## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` は本問では不要（`EXISTS` / `LEFT JOIN ... IS NULL` も登場しない）
* 判定は `project_id` 単位、表示は仕様どおり

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  各プロジェクトについて、そのプロジェクトに所属している従業員の **平均経験年数 (`experience_years`)** を求め、**小数第 2 位まで** に丸めて出力せよ。

* 入力: `{{TABLES_OR_SCHEMAS}}`

  **Project**

  | Column Name | Type |
  | ----------- | ---- |
  | project_id  | int  |
  | employee_id | int  |

  * `(project_id, employee_id)` が主キー
  * `employee_id` は `Employee.employee_id` への外部キー

  **Employee**

  | Column Name      | Type    |
  | ---------------- | ------- |
  | employee_id      | int     |
  | name             | varchar |
  | experience_years | int     |

  * `employee_id` が主キー
  * `experience_years` は NOT NULL

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  | Column Name   | 説明                                  |
  | ------------- | ----------------------------------- |
  | project_id    | プロジェクト ID                           |
  | average_years | そのプロジェクトに属する従業員の平均経験年数。小数第 2 位で四捨五入 |

  * 各 `project_id` につき 1 行
  * 並び順は任意

---

## 2) 最適解（単一クエリ）

> 指示どおり CTE + ウィンドウで組み立てたバージョンです（この問題だけなら単純な `GROUP BY` でも解けますが、ウィンドウ版に寄せます）。

```sql
WITH pre AS (
  SELECT
    p.project_id,
    e.experience_years
  FROM project AS p
  JOIN employee AS e
    ON e.employee_id = p.employee_id
),
win AS (
  SELECT
    project_id,
    AVG(experience_years) OVER (
      PARTITION BY project_id
    ) AS avg_experience_years
  FROM pre
)
SELECT DISTINCT
  project_id,
  ROUND(avg_experience_years::numeric, 2) AS average_years
FROM win;

Runtime 446 ms
Beats 64.43%

```

### （参考）もっと素直な書き方

テンプレートに縛られないなら、現場ではこちらのほうがシンプルで読みやすいです。

```sql
SELECT
  p.project_id,
  ROUND(AVG(e.experience_years)::numeric, 2) AS average_years
FROM project AS p
JOIN employee AS e
  ON e.employee_id = p.employee_id
GROUP BY
  p.project_id;

Runtime 439 ms
Beats 72.22%

```

---

## 3) 要点解説

1. **従業員の紐づけ (`pre` CTE)**
   `project` と `employee` を `employee_id` で結合して、
   「プロジェクト × 従業員の経験年数」の明細を作っています。

   ```sql
   SELECT
     p.project_id,
     e.experience_years
   FROM project AS p
   JOIN employee AS e
     ON e.employee_id = p.employee_id;
   ```

2. **ウィンドウ集計でプロジェクト別の平均を計算 (`win` CTE)**
   `AVG(experience_years) OVER (PARTITION BY project_id)` により、
   各 `project_id` ごとに同じ平均値（ウィンドウ集計結果）が行ごとに付きます。

   ```sql
   AVG(experience_years) OVER (
     PARTITION BY project_id
   ) AS avg_experience_years
   ```

3. **`DISTINCT` で 1 プロジェクト 1 行に整形**
   `win` ではプロジェクトに属する従業員の行数だけ行があるため、
   `SELECT DISTINCT project_id, avg_experience_years` で 1 行に圧縮します。

4. **2 桁丸め (`ROUND`)**
   PostgreSQL で小数第 2 位までに丸めるには `ROUND(numeric, 2)` を使うので、
   ウィンドウの結果を `numeric` にキャストした上で丸めています。

   ```sql
   ROUND(avg_experience_years::numeric, 2) AS average_years
   ```

---

## 4) 計算量（概算）

`n` 行の `Project`、`m` 行の `Employee` を想定します。

* `JOIN`（`employee_id` にインデックスあり前提で hash join / nested loop）
  → おおよそ **O(n + m)** 〜 **O(n log m)** 程度
* ウィンドウ関数 `AVG() OVER (PARTITION BY project_id)`
  → プロジェクトごとの件数を `n_g` とすると、**O(Σ n_g)**（内部ソートがあっても `project_id` での 1 回）
* 全体としてはインデックスが効いていれば **ほぼ線形 O(n + m)** 近辺に収まる想定

この規模の問題（LeetCode レベル）なら、どちらの書き方でも十分高速です。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Project<br/>project_id, employee_id]
  B[Employee<br/>employee_id, experience_years]
  C[CTE pre<br/>project_id + experience_years]
  D["CTE win<br/>AVG(...) OVER (PARTITION BY project_id)"]
  E["SELECT DISTINCT<br/>project_id, ROUND(avg, 2)"]
  F[出力<br/>project_id, average_years]

  A -->|JOIN employee_id| C
  B -->|JOIN employee_id| C
  C --> D
  D --> E
  E --> F
```

こんな感じで、「JOIN → プロジェクトごとに平均 → 丸めて出力」という流れになっています。

結論からいうと、

* **クエリとしてはすでにほぼ最適**
* 446ms vs 439ms は **誤差レベル（実行環境ノイズ）**
* 改善するとしたら **「アルゴリズム」ではなく「シンプルさと無駄の削減」**

くらいの話になります。

---

## 1. 2つのクエリの評価

### ① CTE + ウィンドウ版

```sql
WITH pre AS (
  SELECT
    p.project_id,
    e.experience_years
  FROM project AS p
  JOIN employee AS e
    ON e.employee_id = p.employee_id
),
win AS (
  SELECT
    project_id,
    AVG(experience_years) OVER (
      PARTITION BY project_id
    ) AS avg_experience_years
  FROM pre
)
SELECT DISTINCT
  project_id,
  ROUND(avg_experience_years::numeric, 2) AS average_years
FROM win;
```

* `AVG(...) OVER (PARTITION BY ...)` で **全行に同じ平均値を付与**
* その後 `DISTINCT` で重複を 1 行に畳んでいる
  → **機能的には正しい** ですが、「1プロジェクト1行」が欲しいだけなので、
  *一度ウィンドウで複製してから DISTINCT で消す* という遠回りになっています。

### ② GROUP BY 版

```sql
SELECT
  p.project_id,
  ROUND(AVG(e.experience_years)::numeric, 2) AS average_years
FROM project AS p
JOIN employee AS e
  ON e.employee_id = p.employee_id
GROUP BY
  p.project_id;
```

* こちらは **素直に 1 グループ 1 行を直接作っている**
* 論理的にも物理的にも処理フローが短く、プランナーも最適化しやすい

実行時間の差が小さいのは、PostgreSQL のプランナーが賢くて、
ウィンドウ + DISTINCT も内部的にかなりうまく畳んでくれている可能性が高いです。

---

## 2. 実行時間 446ms vs 439ms について

LeetCode系のランタイムは

* 同じクエリでも **何度も実行すれば普通に ±10〜20ms くらい揺れる**
* 他ユーザーの負荷や環境状態に左右される

ので、**446ms vs 439ms は完全にノイズ範囲** と思って問題ないです。

「Beats 64% → 72%」も、プラットフォーム内での相対比較なので、

* DBエンジンが違う（MySQL / Oracle など）
* インデックス前提が違う
* 言語実装やドライバのオーバーヘッドの差

などいろいろ混ざっています。
PostgreSQL 的には、今の GROUP BY クエリで十分「いい線」です。

---

## 3. ほんの少しだけ直せるところ（PostgreSQL的に）

### 3.1 不要なキャストを削る

PostgreSQL では

* `AVG(int)` の戻り値は **すでに `numeric`**

なので、実は `::numeric` は不要です。

```sql
SELECT
  p.project_id,
  ROUND(AVG(e.experience_years), 2) AS average_years
FROM project AS p
JOIN employee AS e
  ON e.employee_id = p.employee_id
GROUP BY
  p.project_id;
```

これで意味はまったく同じです。
（性能差はほとんど出ませんが、「余計なことをしない」という意味で綺麗。）

### 3.2 `USING` で少しだけ読みやすく

PostgreSQL では `JOIN ... USING (...)` が使えるので、個人的にはこの書き方も好きです。

```sql
SELECT
  p.project_id,
  ROUND(AVG(e.experience_years), 2) AS average_years
FROM project AS p
JOIN employee AS e USING (employee_id)
GROUP BY
  p.project_id;

Runtime 449 ms
Beats 61.92%

```

これも性能はほぼ同じですが、

* 結合条件が「キー1本」のときに読みやすく
* `ON p.employee_id = e.employee_id` という重複記述が消える

というメリットがあります。

---

## 4. チューニングの「本質的な余地」はあるか？

この問題の構造だと、DB側でできることはだいたい以下の2点に集約されます。

1. **JOIN をいかに効率よく行うか**
2. **project_id ごとに平均を出すための集約をいかに効率よく行うか**

そして、この前提では：

* `Employee.employee_id` は PK → **インデックスあり**
* `Project(project_id, employee_id)` も PK → **(project_id, employee_id) インデックスあり**
* `GROUP BY project_id` は、すでにインデックスの先頭列

なので、実行計画としてもかなり恵まれていて、
**これ以上の実行時間短縮は「DBチューニング」領域（メモリ・並列度・統計情報など）**になります。
LeetCode のような環境ではそこは触れないので、クエリレベルでの改善余地はほぼありません。

---

## 5. まとめ

* **CTE + ウィンドウ版**

  * ロジックは綺麗だが、この問題に対しては処理が遠回り気味
* **GROUP BY 版**

  * 問題にもっとも素直で、現場でもこの形が「ベストプラクティス」
  * 実行時間もわずかに良く、プランナーも扱いやすい

最終形としては、これを採用でOKだと思います：

```sql
SELECT
  p.project_id,
  ROUND(AVG(e.experience_years), 2) AS average_years
FROM project AS p
JOIN employee AS e USING (employee_id)
GROUP BY
  p.project_id;
```

もし次のステップとして「さらに一段上の最適化感覚」を鍛えるなら、

* `EXPLAIN (ANALYZE, BUFFERS)` を見て、
* `HashAggregate` / `GroupAggregate` / 使われているインデックス
* 実行時間の内訳（JOIN が重いのか、集約が重いのか）

を読む練習をすると、実務寄りのスキルアップになります 💪